#  We Rate Dogs

In [10]:
#Importing our necessary libraries
import pandas as pd
import requests
import tweepy

In [11]:
#reading in our twitter archive dataset
tac = pd.read_csv('twitter-archive-enhanced.csv')
tac.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


In [12]:
#gathering our image prediction data and then reading it into a dataframe
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)
open('image-predictions.tsv', 'wb').write(r.content)

335079

In [13]:
#read our image predictions file into a datafram
img_pred = pd.read_csv('image-predictions.tsv',  sep='\t')
img_pred.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [17]:
import tweepy

auth = tweepy.OAuthHandler('cDB0XXvmcuasGpYsdwZmNJA1p', 'H8k47M8ANuT6t2nT1GNCqmqAkCccv7ZBKLjlM0iVlqiK7CEOUC')
auth.set_access_token('1139543476083798016-jxHnoF9LgqzMA0klY4Y1FTbKKnlnrY', 'sHszTJ0f57kJjfolJJXOkp2umtG6kVESm41qDAYr4Udtr')

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

In [20]:
# Get the User object for twitter...
user = api.get_user('dog_rates')

In [21]:
print(user.screen_name)
print(user.followers_count)
for friend in user.friends():
   print(friend.screen_name)

dog_rates
8142663
ChronicleBooks
IvePetThatDog
TheVFoundation
FixYourBiz
hess_clint
Summersdale
AndrewsMcMeel
skyhorsepub
dog_feelings
unwise
mackemode
dogfather
